In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
import numpy as np

In [5]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import PIL

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Conv2D,Dropout
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adagrad,Adam
import numpy as np
from tensorflow.keras.models import load_model  



In [14]:
IMG_SIZE = 224
def load_imgs(df):
    imgs = np.ndarray(shape = (len(df), IMG_SIZE, IMG_SIZE,3), dtype = np.float32)
    for ii in range(len(df)):
        file = df.img.values[ii]
        img = load_img(data_path+'images/'+file, target_size=(IMG_SIZE, IMG_SIZE),color_mode='rgb')
        img = img_to_array(img)/255
        imgs[ii] = img
    return imgs

In [8]:
cd CUB_200_2011

C:\Users\lenovo\CUB_200_2011


In [9]:
pwd

'C:\\Users\\lenovo\\CUB_200_2011'

In [10]:
data_path = 'C:\\Users\\lenovo\\CUB_200_2011\\'

In [11]:
df_images = pd.read_csv(data_path+'images.txt', 
                        sep = ' ',header = None, 
                        names = ['img_num','img'])

In [12]:
df_labels = pd.read_csv(data_path+'image_class_labels.txt', 
                        sep = ' ',header = None, 
                        names = ['img_num','class_id'])
df_classes = pd.read_csv(data_path+'classes.txt', 
                         sep = ' ', header = None, 
                         names = ['class_id','bird_class'])
df_split = pd.read_csv(data_path +'train_test_split.txt', 
                       sep = ' ', header = None, 
                       names = ['img_num','dataset'])
df = pd.merge(df_images, df_labels, on = 'img_num', how = 'inner')
df = pd.merge(df, df_classes, on = 'class_id',how = 'inner')
df = pd.merge(df, df_split, on = 'img_num',how = 'inner')

In [13]:
warbler_class = ['158.Bay_breasted_Warbler',
       '159.Black_and_white_Warbler', '160.Black_throated_Blue_Warbler',
       '161.Blue_winged_Warbler', '162.Canada_Warbler',
       '163.Cape_May_Warbler', '164.Cerulean_Warbler',
       '165.Chestnut_sided_Warbler', '166.Golden_winged_Warbler',
       '167.Hooded_Warbler', '168.Kentucky_Warbler',
       '169.Magnolia_Warbler', '170.Mourning_Warbler',
       '171.Myrtle_Warbler', '172.Nashville_Warbler',
       '173.Orange_crowned_Warbler', '174.Palm_Warbler',
       '175.Pine_Warbler', '176.Prairie_Warbler',
       '177.Prothonotary_Warbler', '178.Swainson_Warbler',
       '179.Tennessee_Warbler', '180.Wilson_Warbler',
       '181.Worm_eating_Warbler', '182.Yellow_Warbler'
                ]

In [14]:
sparrow_class = [
    '113.Baird_Sparrow',
     '114.Black_throated_Sparrow',
    '115.Brewer_Sparrow',
    '116.Chipping_Sparrow',
    '117.Clay_colored_Sparrow',
    '118.House_Sparrow',
    '119.Field_Sparrow',
    '120.Fox_Sparrow',
    '121.Grasshopper_Sparrow',
    '122.Harris_Sparrow',
    '123.Henslow_Sparrow',
    '124.Le_Conte_Sparrow',
    '125.Lincoln_Sparrow',
    '126.Nelson_Sharp_tailed_Sparrow',
    '127.Savannah_Sparrow',
    '128.Seaside_Sparrow',
    '129.Song_Sparrow',
    '130.Tree_Sparrow',
    '131.Vesper_Sparrow'
    '132.White_crowned_Sparrow',
    '133.White_throated_Sparrow',
]

In [15]:
wern_class=[
 '193.Bewick_Wren',
 '194.Cactus_Wren',
 '195.Carolina_Wren',
 '196.House_Wren',
 '197.Marsh_Wren',
 '198.Rock_Wren',
 '199.Winter_Wren'
]

In [16]:
gull_class=[
 '059.California_Gull',
 '060.Glaucous_winged_Gull',
 '061.Heermann_Gull',
 '062.Herring_Gull',
 '063.Ivory_Gull',
 '064.Ring_billed_Gull',
 '065.Slaty_backed_Gull',
 '066.Western_Gull'
]

In [18]:
warbler_df=df.loc[df.bird_class.isin(warbler_class)]
sparrow_df=df.loc[df.bird_class.isin(sparrow_class)]
wern_df=df.loc[df.bird_class.isin(wern_class)]
gull_df=df.loc[df.bird_class.isin(gull_class)]

In [19]:
warbler_x_train = load_imgs(warbler_df)
sparrow_x_train = load_imgs(sparrow_df)
wern_x_train = load_imgs(wern_df)
gull_x_train = load_imgs(gull_df)

In [15]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))
model_1 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False) # last layer trainable
model_2 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)#last two layers trainable 
model_3 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False) # last three layers trainable 
model_4 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False) #last four layers trainable

Using TensorFlow backend.


In [16]:
model_all = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False) #all layers trainable

In [22]:
x_old_train=warbler_x_train

In [43]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import Model
import sklearn

In [44]:
def add_fc(model):
    aux=model.output
    tail=GlobalAveragePooling2D()(aux)
    tail=Dense(2,activation='softmax')(tail)
    final_model=Model(inputs=model.input, outputs=tail)
    return final_model

In [158]:
model_1=keras.models.load_model('model_3.h5')

In [159]:
for layer in model_1.layers[:-1]:
    layer.trainable=False

In [25]:
warbler_x_train=np.load('warbler_x_train.npy')
sparrow_x_train=np.load('sparrow_x_train.npy')
wern_x_train=np.load('wern_x_train.npy')
gull_x_train=np.load('gull_x_train.npy')

In [167]:
x_old_train=warbler_x_train

In [168]:
x_old_train=np.concatenate([x_old_train,sparrow_x_train],axis=0)

In [169]:
y_old_pred=np.zeros((x_old_train.shape[0],2))

In [170]:
for i in range (0,warbler_x_train.shape[0]-1):
    y_old_pred[i][0]=1
for i in range (warbler_x_train.shape[0],y_old_pred.shape[0]):
    y_old_pred[i][1]=1

In [171]:
for i in y_old_pred:
    print (i)

[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]


[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]


[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]


In [172]:
def shuffle_in_unison(a,b):

    assert len(a)==len(b)
    c = np.arange(len(a))
    np.random.shuffle(c)

    return a[c],b[c]

In [173]:
(x_old_train,y_old_pred)=shuffle_in_unison(x_old_train,y_old_pred)

In [174]:
model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [176]:
model_1.fit(x=x_old_train,y=y_old_pred,epochs=10)

Train on 2614 samples
Epoch 1/10
2614/2614 [==============================] - 433s 166ms/sample - loss: 0.5081 - accuracy: 0.7414
Epoch 2/10
2614/2614 [==============================] - 363s 139ms/sample - loss: 0.3648 - accuracy: 0.8416
Epoch 3/10
2614/2614 [==============================] - 368s 141ms/sample - loss: 0.3231 - accuracy: 0.8646
Epoch 4/10
2614/2614 [==============================] - 361s 138ms/sample - loss: 0.3062 - accuracy: 0.8642
Epoch 5/10
2614/2614 [==============================] - 360s 138ms/sample - loss: 0.2857 - accuracy: 0.8818
Epoch 6/10
2614/2614 [==============================] - 364s 139ms/sample - loss: 0.2772 - accuracy: 0.8868
Epoch 7/10
2614/2614 [==============================] - 362s 138ms/sample - loss: 0.2747 - accuracy: 0.8917
Epoch 8/10
2614/2614 [==============================] - 366s 140ms/sample - loss: 0.2636 - accuracy: 0.9013
Epoch 9/10
2614/2614 [==============================] - 363s 139ms/sample - loss: 0.2431 - accuracy: 0.9051
Epoch 

In [177]:
model_1.save('model.h5')

In [178]:
def add_class(model):
    nodes=model.layers[-1].output_shape[-1]
    weights_bak=model.layers[-1].get_weights()
    model.layers.pop()
    model.layers.pop()
    aux=model.get_layer('mixed10').output  
    tail=GlobalAveragePooling2D()(aux)
    tail=Dense(nodes+1,activation='softmax')(tail)
    new_model=Model(inputs=model.input, outputs=tail)
    weights_new=new_model.layers[-1].get_weights()
    weights_new[0][:,:-1]=weights_bak[0]
    weights_new[1][:-1]=weights_bak[1]
    
    weights_new[0][:,-1]=np.mean(weights_bak[0],axis=1)
    weights_new[1][-1]=np.mean(weights_bak[0])
    new_model.layers[-1].set_weights(weights_new)
    return new_model

In [179]:
model_1=keras.models.load_model('model.h5')

In [180]:
y_old_label=y_old_pred

In [181]:
y_old_pred=model_1.predict(wern_x_train)


In [182]:
np.save('y_old_pred.npy',y_old_pred)

In [183]:
y_soft=np.append(y_old_pred,np.zeros((y_old_pred.shape[0],1)),axis=1)

In [184]:
np.save('y_soft.npy',y_soft)

In [185]:
y_hard=np.zeros(y_soft.shape)

In [186]:
for i in range (0,y_hard.shape[0]):
    y_hard[i][2]=1

In [187]:
y_hard

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [188]:
import tensorflow.keras.backend as k
from tensorflow.keras.losses import categorical_crossentropy as logloss

In [189]:
y_comb_label=np.concatenate([y_hard,y_soft],axis=1)

In [239]:
def custom_loss(y_true,y_pred):
    y_hard,y_soft = y_true[: , :4], y_true[: , 4:]
    loss = logloss(y_hard,y_pred) + logloss(y_soft, y_pred)
    return loss

In [191]:
model_1=add_class(model_1)

In [192]:
model_1.compile(loss=custom_loss,optimizer='adam',metrics=['accuracy'])

In [193]:
model_1.fit(x=wern_x_train,y=y_comb_label,epochs=10)

Train on 419 samples
Epoch 1/10
419/419 [==============================] - 87s 208ms/sample - loss: 2.1582 - accuracy: 0.6014
Epoch 2/10
419/419 [==============================] - 58s 138ms/sample - loss: 2.0242 - accuracy: 0.6516
Epoch 3/10
419/419 [==============================] - 59s 141ms/sample - loss: 1.9740 - accuracy: 0.6563
Epoch 4/10
419/419 [==============================] - 59s 140ms/sample - loss: 1.9369 - accuracy: 0.6706
Epoch 5/10
419/419 [==============================] - 58s 138ms/sample - loss: 1.9240 - accuracy: 0.6802
Epoch 6/10
419/419 [==============================] - 58s 138ms/sample - loss: 1.8901 - accuracy: 0.6468
Epoch 7/10
419/419 [==============================] - 63s 150ms/sample - loss: 1.9013 - accuracy: 0.6993
Epoch 8/10
419/419 [==============================] - 58s 138ms/sample - loss: 1.8491 - accuracy: 0.7184
Epoch 9/10
419/419 [==============================] - 57s 137ms/sample - loss: 1.8354 - accuracy: 0.6754
Epoch 10/10
419/419 [=============

In [200]:
y_test=np.append(y_old_label,np.zeros((y_old_label.shape[0],1)),axis=1)
for i in y_test:
    print (i)

[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]

[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]

[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]

[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[0. 1. 0.]

In [201]:
y_test

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [207]:
y_pred=model_1.predict(x_old_train)

In [208]:
from sklearn.metrics import f1_score

In [210]:
yp1=np.argmax(y_pred,axis=1)

In [213]:
yp2=np.argmax(y_test,axis=1)

In [220]:
f1_score(yp2, yp1, average='weighted')

0.5799811831428839

In [221]:
model_1.save('model_1.h5')

In [222]:
y_gull_pred=model_1.predict(gull_x_train)

In [223]:
y_gull_pred

array([[0.63745564, 0.01635784, 0.34618646],
       [0.6827364 , 0.00891278, 0.30835083],
       [0.1165327 , 0.5492754 , 0.33419192],
       ...,
       [0.4644852 , 0.08387878, 0.451636  ],
       [0.6752935 , 0.01304117, 0.31166527],
       [0.6360048 , 0.0103494 , 0.35364577]], dtype=float32)

In [224]:
model_1=add_class(model_1)

In [227]:
y_gull_hard=np.zeros((gull_x_train.shape[0],4))

In [232]:
for i in range (0,y_gull_hard.shape[0]):
    y_gull_hard[i][3]=1

In [234]:
y_gull_soft=np.append(y_gull_pred,np.zeros((y_gull_pred.shape[0],1)),axis=1)

In [235]:
y_gull_soft

array([[0.63745564, 0.01635784, 0.34618646, 0.        ],
       [0.6827364 , 0.00891278, 0.30835083, 0.        ],
       [0.1165327 , 0.5492754 , 0.33419192, 0.        ],
       ...,
       [0.4644852 , 0.08387878, 0.45163599, 0.        ],
       [0.67529351, 0.01304117, 0.31166527, 0.        ],
       [0.63600481, 0.0103494 , 0.35364577, 0.        ]])

In [236]:
y_comb_label=np.concatenate([y_gull_hard,y_gull_soft],axis=1)

In [240]:
model_1.compile(loss=custom_loss,optimizer='adam',metrics=['accuracy'])

In [241]:
model_1.fit(x=gull_x_train,y=y_comb_label,epochs=10)

Train on 469 samples
Epoch 1/10
469/469 [==============================] - 95s 203ms/sample - loss: 2.4957 - accuracy: 0.8124
Epoch 2/10
469/469 [==============================] - 67s 143ms/sample - loss: 2.3933 - accuracy: 0.8998
Epoch 3/10
469/469 [==============================] - 65s 138ms/sample - loss: 2.3514 - accuracy: 0.9126
Epoch 4/10
469/469 [==============================] - 67s 142ms/sample - loss: 2.3390 - accuracy: 0.9488
Epoch 5/10
469/469 [==============================] - 65s 139ms/sample - loss: 2.3179 - accuracy: 0.9829
Epoch 6/10
469/469 [==============================] - 66s 140ms/sample - loss: 2.3108 - accuracy: 0.9829
Epoch 7/10
469/469 [==============================] - 65s 139ms/sample - loss: 2.3073 - accuracy: 0.9723
Epoch 8/10
469/469 [==============================] - 66s 141ms/sample - loss: 2.3052 - accuracy: 0.9829
Epoch 9/10
469/469 [==============================] - 65s 139ms/sample - loss: 2.2937 - accuracy: 0.9872
Epoch 10/10
469/469 [=============

In [242]:
y_pred=model_1.predict(wern_x_train)

In [246]:
p=wern_x_train.shape[0]
yp1=np.zeros((1,p))

In [254]:
for i in range(0,yp1.shape[1]):
    yp1[0][i]=2

In [256]:
yp2=np.argmax(y_pred,axis=1)

In [258]:
y_pred

array([[0.16953626, 0.00429319, 0.23169595, 0.5944746 ],
       [0.17555127, 0.00932954, 0.26675513, 0.54836416],
       [0.01420809, 0.10381459, 0.20168966, 0.68028766],
       ...,
       [0.16139139, 0.00635459, 0.20500113, 0.6272529 ],
       [0.16537347, 0.00405334, 0.2458114 , 0.5847618 ],
       [0.36643732, 0.00122694, 0.23122971, 0.40110603]], dtype=float32)